In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
#定義常用函數
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
#準備訓練資料
x_data = np.linspace(-0.5, 0.5, 500, dtype = np.double)[:,np.newaxis]
y_data = np.power(x_data, 1) + np.random.normal(0, 0.01, x_data.shape)

In [ ]:
#學習速率
learning_rate = 0.001

def allShape(list):
    for arr in list:
        print(arr.shape)

In [ ]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(1, y) for y in sizes[1:]]
        self.weights = [np.random.randn(x, y)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        return self.feedforwardRecord(a)[-1]
    
    def feedforwardRecord(self, a):
        x = [np.array([a])]
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(a, w) + b)
            x.append(a)
        return x
        
    def backpropagationWith(self, x, y):
        
        delta_out = []
        delta_in = x[-1] - y #cross entropy loss
        
        dbiases = [np.dot(np.transpose(x[-1]), delta_in)]
        dweights = [delta_in]
        
        for b_idx in range(self.num_layers - 2):
            
            idx = self.num_layers - b_idx - 2
            
            delta_out = np.dot(delta_in, np.transpose(self.weights[idx]))
            
            c_x = x[idx - 1]
            c_w = self.weights[idx - 1]
            c_sum = np.dot(c_x, c_w)
            
            delta_in = delta_out * sigmoid_prime(c_sum)
            db = delta_in
            dw = np.dot(np.transpose(c_x), delta_in)
            
            dbiases.insert(0, db)
            dweights.insert(0, dw)
        
        return (dbiases, dweights)
    
    def updateWith(self, dbs, dws):
        
        new_b = []
        for b, db in zip(self.biases, dbs):
            new_b.append(b - learning_rate * db)
        
        new_w = []
        for w, dw in zip(self.weights, dws):
            new_w.append(w - learning_rate * dw)
        
        self.biases = new_b
        self.weights = new_w

In [ ]:
myNN = Network([1, 2, 3, 1])
#print(myNN.biases, myNN.weights)

In [ ]:
for _ in range(100000):
    idx = np.random.randint(x_data.size)
    re = myNN.feedforwardRecord(x_data[idx])
    (dbs, dws) = myNN.backpropagationWith(re, y_data[idx])    
    myNN.updateWith(dbs, dws)

In [ ]:
new_y = np.zeros(y_data.shape)

index = 0
while index < x_data.size:
    x = x_data[index]
    new_y[index] = myNN.feedforward(x)
    index = index + 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(x_data, y_data,'y-', lw = 2) #原本
ax.plot(x_data, new_y,'r-', lw = 2) #推測

plt.show()